In [1]:
import sys

!which {sys.executable}

/ut3/jerome/miniconda3/envs/wakai/bin/python


In [2]:
import sys
sys.path.insert(0, '../code')

# TOI catalog

In [4]:
from catalog import get_tois

tois = get_tois(clobber=False)
# tois.columns.tolist()

In [5]:
tois.tail()

,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments,ra_deg,dec_deg,GaiaDR3_exofop,GaiaDR3_vizier
6471,271052158,7461.01,NaN,5,5,5,5,5,5,5,...,0.111650,"15,40,41,54,55,74,75",2025-06-05,2025-06-05,2025-07-22 14:39:41,Kepler-314 c,294.674167,42.075603,NaN,NaN
6472,298897095,7462.01,NaN,3,4,3,3,4,4,4,...,NaN,"74,75",2025-06-05,2025-06-05,2025-07-22 14:39:41,KOI 141.01; SPOC is assuming solar radius,288.038208,50.651475,NaN,NaN
6473,274198407,7463.01,NaN,5,5,5,5,5,5,5,...,0.143742,"41,54,55,74,75",2025-06-05,2025-06-05,2025-07-22 14:39:41,Kepler-318 b,298.482833,46.793636,NaN,NaN
6474,251090642,7464.01,TIC 251090642.01,3,4,3,3,1,4,4,...,0.020273,"47,74",2025-06-05,2025-06-05,2025-07-22 14:39:41,also a CTOI from Daniel Vogl,139.793750,53.594558,NaN,NaN
6475,167804011,7465.01,TIC 167804011.01,3,4,3,3,1,4,4,...,0.116703,90,2025-07-17,2025-07-17,2025-07-22 12:04:06,egress of one transit right after downlink gap,162.431083,-23.349503,NaN,NaN


In [6]:
tois[tois.Comments.apply(lambda x: 'young' in str(x))][['TOI','Comments']].values

array([[942.01,
        'Probable young K dwarf. Transit is believable enough. Maybe also a signal at 10.15 days with Tc=1447?'],
       [3604.01,
        'V-shaped; found in faint-star QLP search; no stellar radius; possible young or evolved K star'],
       [5743.01,
        'radius possibly underestimated; potentially young host star'],
       [5956.01,
        'V-shaped; large; young host star; ~5 sigma odd-even in SPOC s57'],
       [6258.01,
        'low SNR; asymmetric transit shape; flaring possibly young host'],
       [6678.01,
        'rotational modulation at 7-8 days may indicate young host star']],
      dtype=object)

In [7]:
tois.shape

(6476, 66)

# cross-match TOI with Gaia DR3 using `astroquery.xmatch`

In [8]:
import astropy.units as u
from astroquery.xmatch import XMatch
import pandas as pd

gaia_table = XMatch.query(
    cat1=open('../data/TOIs.csv'),
    cat2="vizier:I/355/gaiadr3",  # The Vizier identifier for Gaia DR3
    max_distance=1 * u.arcsec,    # Set the maximum matching radius
    colRA1="ra_deg",  # Column name for Right Ascension in TIC results
    colDec1="dec_deg" # Column name for Declination in TIC results
)
gaia_df = gaia_table.to_pandas()

# results to 6237 matches
matches = gaia_df.copy()

# Apply magnitude filtering criteria
# For main-sequence stars, the difference between TESS T and Gaia G is around 0–0.5 magnitudes
# Redder stars might have T - G values closer to 1–2 magnitudes.
# For giants or highly reddened stars, the difference could be larger.
if False:
    # results to 5971 matches
    matches = []
    for i, row in gaia_df.iterrows():
        teff=row['Stellar Eff Temp (K)']
        if teff<3500:
            magnitude_threshold = 2
        elif (teff>3500) and (teff<6000):
            magnitude_threshold = 1
        else:
            magnitude_threshold = 1.5
        if abs(row['Gmag'] - row['TESS Mag']) < magnitude_threshold:
            matches.append(row)
    matches = pd.concat(matches, axis=1).T
if False:
    # results to 5700 matches
    magnitude_threshold = 1
    matches = gaia_df[abs(gaia_df['Gmag'] - gaia_df['TESS Mag']) < magnitude_threshold]
if False:
    # results to 4878 matches
    teff_threshold = 1000
    matches = gaia_df[abs(gaia_df['Teff'] - gaia_df['Stellar Eff Temp (K)']) < teff_threshold]
matches.tail()

Could not import regions, which is required for some of the functionalities of this module.


,angDist,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,...,B_AG,E(BP-RP),b_E(BP-RP),B_E(BP-RP),Lib,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000
7755,0.039712,271052158,7461.01,NaN,5,5,5,5,5,5,...,0.1218,0.0602,0.0558,0.0640,MARCS,294.674111,42.075579,0.159579,0.191329,0.1048
7756,0.027989,298897095,7462.01,NaN,3,4,3,3,4,4,...,0.0036,0.0007,0.0001,0.0020,PHOENIX,288.038299,50.651556,0.612471,0.419156,0.0106
7757,0.001909,274198407,7463.01,NaN,5,5,5,5,5,5,...,0.1827,0.0924,0.0885,0.0973,MARCS,298.482822,46.793623,0.187789,0.209036,0.1047
7758,0.032425,251090642,7464.01,TIC 251090642.01,3,4,3,3,1,4,...,0.4923,0.2877,0.2828,0.2931,MARCS,139.793957,53.594645,0.260186,0.244903,0.0733
7759,0.042709,167804011,7465.01,TIC 167804011.01,3,4,3,3,1,4,...,NaN,NaN,NaN,NaN,NaN,162.431299,-23.349510,0.575813,0.539205,-0.2146


In [9]:
matches.shape

(7760, 204)

In [10]:
matches['GaiaDR3'] = matches['DR3Name'].apply(lambda x: int(x.split()[-1]))
matches.to_csv('../data/TOIs_GaiaDR3_xmatch.csv', index=False)

In [11]:
idx = tois.TOI.isin(matches.TOI.unique())
tois[~idx].shape

(15, 66)

In [12]:
tois[~idx].to_csv('../data/TOIs_GaiaDR3_xmatch_missing.csv', index=False)